# WACC Calculator

This notebook finds the WACC (weighted average cost of capital) for a given ticker based on the capital structure of its market peers (i.e. peers' equity values, debt values, corporate tax rates, and volatilities).

Because WACC is highly subjective, differing by a couple hundred basis points depending on the analyst, in this notebook we simply find a reasonable middle ground to use as a baseline in future code.

### Define Imports & Universe

In [32]:
import json, os
from pprint import pprint

from src.wacc.model import compute_wacc, peer_unlevered_beta
from src.wacc.inputs import fetch_peer_rows_yf, build_baselines_for_wacc

TICKER = "NEE"
PEERS = ["DUK", "SO", "AEP", "D"]


### Fetch Peer Data & Setup Baselines

In [33]:
# poll financial data for peer companies using yahoo finance
peer_rows = fetch_peer_rows_yf(PEERS)
len(peer_rows), peer_rows[:2]

# build baseline parameters for WACC calculations based on this peer data
params = build_baselines_for_wacc(ticker=TICKER, peer_rows=peer_rows)
pprint(params)


{'debt_pricing': {'spread': 0.02},
 'mrp': 0.055,
 'rf': 0.045,
 'target_de_ratio': 0.736799840085523,
 'tax': 0.21}


### Compute WACC for Given Ticker

In [34]:
out = compute_wacc(
    peers=peer_rows,          # infer asset beta from peers, then relever to target_de_ratio
    n_draws=10_000,
    sd_beta=0.15, sd_rf=0.003, sd_mrp=0.01, sd_codebt=0.005,
    random_state=42,
    strict=True,              # fail if anything essential is missing
    **params                  # rf, mrp, tax, target_de_ratio, debt_pricing
)

pprint({k: out[k] for k in ["wacc","beta","coeq","codebt","tax","de_ratio"]})
pprint(out.get("wacc_pct"))


{'beta': 0.601,
 'codebt': 0.065,
 'coeq': 0.078055,
 'de_ratio': 0.736799840085523,
 'tax': 0.21,
 'wacc': 0.0667259802273387}
{'p25': 0.062280835909766644,
 'p5': 0.05692120570860097,
 'p50': 0.06636870024947045,
 'p75': 0.07085505003193586,
 'p95': 0.07784621283156022}


### Save Artifacts

In [35]:
ART_DIR = "../artifacts"; os.makedirs(ART_DIR, exist_ok=True)
with open(os.path.join(ART_DIR, f"{TICKER}_wacc.json"), "w") as f:
    json.dump(out, f, indent=2)
